In [46]:
!pip install requests pandas numpy

In [3]:
import requests
import pandas as pd
import numpy as np
import json

In [4]:
CURRENT_QUIZ_URL = "https://www.jsonkeeper.com/b/LLQT"
HISTORICAL_QUIZ_URL = "https://api.jsonserve.com/XgAgFJ"

In [5]:
def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

In [6]:
current_quiz_data = fetch_data(CURRENT_QUIZ_URL)
historical_quiz_data = fetch_data(HISTORICAL_QUIZ_URL)

In [16]:
def calculate_overall_accuracy(quiz_history):
    accuracies = [float(quiz.get('accuracy', '0 %').strip('%')) for quiz in quiz_history]
    return np.mean(accuracies) if accuracies else 0

def analyze_topic_performance(quiz_history):
    topic_performances = {}
    for quiz in quiz_history:
        topic = quiz.get('quiz', {}).get('topic', 'Unknown')
        accuracy = float(quiz.get('accuracy', '0 %').strip('%'))
        topic_performances[topic] = accuracy
    return topic_performances

In [17]:
def analyze_difficulty_performance(quiz_history):
    difficulty_performances = {}
    for quiz in quiz_history:
        # Use quiz type or other available difficulty indicators
        difficulty = quiz.get('type', 'Unknown')
        accuracy = float(quiz.get('accuracy', '0 %').strip('%'))
        difficulty_performances[difficulty] = accuracy
    return difficulty_performances

In [18]:
# persona analyze metrics
def generate_student_persona(overall_accuracy, topic_performance):
    if overall_accuracy >= 80:
        persona = "The Scholar"
    elif overall_accuracy >= 60:
        persona = "The Emerging Talent"
    else:
        persona = "The Potential Learner"

    if topic_performance:
        weakest_topic = min(topic_performance, key=topic_performance.get)
        strongest_topic = max(topic_performance, key=topic_performance.get)
        return f"{persona} with strengths in {strongest_topic} and challenges in {weakest_topic}"
    return persona

In [19]:
def generate_recommendations(topic_performance, difficulty_breakdown, student_persona):
    recommendations = []

    if topic_performance:
        weakest_topic = min(topic_performance, key=topic_performance.get)
        recommendations.append(f"Focus intensively on {weakest_topic} to improve overall performance")

    if difficulty_breakdown:
        lowest_difficulty = min(difficulty_breakdown, key=difficulty_breakdown.get)
        recommendations.append(f"Practice more {lowest_difficulty} type questions to build confidence")

    if "Scholar" in student_persona:
        recommendations.append("Explore advanced problem-solving techniques to maintain excellence")
    elif "Emerging Talent" in student_persona:
        recommendations.append("Develop consistent study habits and targeted practice strategies")
    else:
        recommendations.append("Build a structured learning plan with gradual complexity increase")

    return recommendations

In [35]:
def analyze_performance(user_id, historical_quiz_data):
    # this is for user's quiz history
    user_history = [quiz for quiz in historical_quiz_data if quiz.get('user_id') == user_id]

    # Check karo if empty
    if not user_history:
        print(f"No quiz history found for user ID: {user_id}")
        return None

    # Calculate metrics
    overall_accuracy = calculate_overall_accuracy(user_history)
    topic_performance = analyze_topic_performance(user_history)
    difficulty_breakdown = analyze_difficulty_performance(user_history)

    student_persona = generate_student_persona(overall_accuracy, topic_performance)
    recommendations = generate_recommendations(topic_performance, difficulty_breakdown, student_persona)

    return {
        'overall_accuracy': overall_accuracy,
        'topic_performance': topic_performance,
        'difficulty_breakdown': difficulty_breakdown,
        'student_persona': student_persona,
        'recommendations': recommendations
    }

# Debug
user_ids = set(quiz.get('user_id') for quiz in historical_quiz_data)
print("Available User IDs:", list(user_ids)[:5])

# Choose a user ID from the available list
if user_ids:
    user_id = list(user_ids)[0]
    analysis_results = analyze_performance(user_id, historical_quiz_data)

    if analysis_results:
        print("\nQuiz Performance Analysis Results:")
        print(f"User ID: {user_id}")
        print(f"Overall Accuracy: {analysis_results['overall_accuracy']:.2f}%")

        print("\nTopic Performance:")
        for topic, score in analysis_results['topic_performance'].items():
            print(f"{topic}: {score:.2f}%")

        print("\nDifficulty Breakdown:")
        for difficulty, score in analysis_results['difficulty_breakdown'].items():
            print(f"{difficulty}: {score:.2f}%")

        print(f"\nStudent Persona: {analysis_results['student_persona']}")

        print("\nRecommendations:")
        for recommendation in analysis_results['recommendations']:
            print(f"- {recommendation}")
    else:
        print("Unable to generate analysis.")
else:
    print("No user IDs found in the data.")

Available User IDs: ['YcDFSO4ZukTJnnFMgRNVwZTE4j42']

Quiz Performance Analysis Results:
User ID: YcDFSO4ZukTJnnFMgRNVwZTE4j42
Overall Accuracy: 72.21%

Topic Performance:
Body Fluids and Circulation: 31.00%
Body Fluids and Circulation : 50.00%
Human Reproduction: 38.00%
principles of inheritance and variation : 30.00%
microbes in human welfare: 100.00%
reproductive health : 84.00%
human health and disease : 93.00%
Reproductive Health: 43.00%
Respiration and Gas Exchange: 66.00%

Difficulty Breakdown:
topic: 66.00%

Student Persona: The Emerging Talent with strengths in microbes in human welfare and challenges in principles of inheritance and variation 

Recommendations:
- Focus intensively on principles of inheritance and variation  to improve overall performance
- Practice more topic type questions to build confidence
- Develop consistent study habits and targeted practice strategies


In [36]:
# for specific quiz
def analyze_quiz_performance(quiz_data):
    # performance analysis
    quiz_details = {
        'quiz_id': quiz_data.get('id'),
        'topic': quiz_data.get('quiz', {}).get('topic', 'Unknown'),
        'accuracy': float(quiz_data.get('accuracy', '0 %').strip('%')),
        'score': quiz_data.get('score', 0),
        'correct_answers': quiz_data.get('correct_answers', 0),
        'incorrect_answers': quiz_data.get('incorrect_answers', 0),
        'total_questions': quiz_data.get('total_questions', 0)
    }

    insights = []

    # Accuracy assessment
    if quiz_details['accuracy'] >= 90:
        insights.append("Exceptional performance")
    elif quiz_details['accuracy'] >= 75:
        insights.append("Strong performance")
    elif quiz_details['accuracy'] >= 60:
        insights.append("Moderate performance")
    else:
        insights.append("Needs improvement")

    # Question breakdown
    if quiz_details['total_questions'] > 0:
        correct_percentage = (quiz_details['correct_answers'] / quiz_details['total_questions']) * 100
        insights.append(f"Correct Answers: {correct_percentage:.2f}%")

    # Response map analysis
    response_map = quiz_data.get('response_map', {})
    if response_map:
        total_responses = len(response_map)
        insights.append(f"Total Questions Attempted: {total_responses}")

    return {
        'quiz_details': quiz_details,
        'insights': insights,
        'recommendations': [
            f"Focus on improving performance in {quiz_details['topic']}",
            "Review incorrect answers to understand knowledge gaps",
            "Practice similar difficulty level questions"
        ]
    }

In [40]:
def analyze_specific_quiz(quiz_id, historical_quiz_data):
    # Find the specific quiz
    quiz_data = next((quiz for quiz in historical_quiz_data if quiz.get('id') == quiz_id), None)

    if quiz_data:
        analysis = analyze_quiz_performance(quiz_data)

        print("Quiz Performance Analysis:")
        print("\nQuiz Details:")
        for key, value in analysis['quiz_details'].items():
            print(f"{key.replace('_', ' ').title()}: {value}")

        print("\nInsights:")
        for insight in analysis['insights']:
            print(f"- {insight}")

        print("\nRecommendations:")
        for recommendation in analysis['recommendations']:
            print(f"- {recommendation}")

        return analysis
    else:
        print(f"No quiz found with ID: {quiz_id}")
        return None

In [44]:
quiz_ids = [quiz.get('id') for quiz in historical_quiz_data]
print("Available Quiz IDs:", quiz_ids[:10])

Available Quiz IDs: [336497, 336448, 333330, 333242, 329504, 328488, 328414, 321514, 320963, 320916]


In [45]:
if quiz_ids:
    analyze_specific_quiz(quiz_ids[7], historical_quiz_data)

Quiz Performance Analysis:

Quiz Details:
Quiz Id: 321514
Topic: principles of inheritance and variation 
Accuracy: 30.0
Score: 12
Correct Answers: 3
Incorrect Answers: 7
Total Questions: 59

Insights:
- Needs improvement
- Correct Answers: 5.08%
- Total Questions Attempted: 10

Recommendations:
- Focus on improving performance in principles of inheritance and variation 
- Review incorrect answers to understand knowledge gaps
- Practice similar difficulty level questions
